# Assignment - Game: UNO

## Instructions

This is a **self-contained notebook** - everything you need is here!

### Quick Start
1. **Run all cells** up to Section 4 (this loads the game client)
2. **Implement your solver** in Section 5
3. **Update STUDENT_TOKEN** in Section 6
4. **Play the game** by running Section 6

### What You Need To Do
- Focus ONLY on implementing `my_agent()` function (Section 5)
- You can also use `manual_player_solver` to play manually if you want
- Everything else is provided for you!

### About UNO
UNO is a classic card game:
- Players try to get rid of all their cards first
- Match cards by color or number/symbol
- Special cards: Skip, Reverse, Draw 2, Wild, Wild Draw 4
- Must say "UNO" when you have one card left (automatic)
- Strategic card management and timing is key

---
## Section 1: Setup

**Run this cell (no changes needed)**

In [1]:
import requests
import json
import time
import random
from typing import List, Optional, Tuple, Any, Dict

print("✅ Dependencies imported")

BASE_URL = 'https://ie-aireasoning-gr4r5bl6tq-ew.a.run.app'

print("✅ Configuration loaded")

✅ Dependencies imported
✅ Configuration loaded


---
## Section 2: Game Client Library

**Run this cell (no changes needed)**

In [2]:
class GameClient:
    def __init__(self, base_url: str, token: str, debug: bool = False):
        self.base_url = base_url.rstrip('/')
        self.token = token
        self.debug = debug

    def _make_request(self, endpoint: str, params: dict, max_retries: int = 10) -> dict:
        params['TOKEN'] = self.token
        url = f'{self.base_url}{endpoint}'
 
        for attempt in range(max_retries):
            try:
                if self.debug:
                    print(f"[DEBUG] Request: {endpoint}")
                    print(f"[DEBUG] Params: {params}")

                response = requests.get(url, params=params, timeout=30)

                if self.debug:
                    print(f"[DEBUG] Response [{response.status_code}]: {response.text[:200]}")

                if response.status_code == 200:
                    if response.text:
                        try:
                            return response.json()
                        except (json.JSONDecodeError, ValueError) as e:
                            if self.debug:
                                print(f"[DEBUG] Non-JSON response: {response.text[:100]}")
                            return {}
                    return {}
                else:
                    print(f"⚠️  HTTP {response.status_code}: {response.text[:200]}")

            except requests.exceptions.Timeout:
                print(f"⚠️  Request timeout (attempt {attempt + 1}/{max_retries})")
            except requests.exceptions.RequestException as e:
                print(f"⚠️  Request error: {e} (attempt {attempt + 1}/{max_retries})")
            except Exception as e:
                print(f"⚠️  Unexpected error: {type(e).__name__}: {e} (attempt {attempt + 1}/{max_retries})")

            if attempt < max_retries - 1:
                time.sleep(1)

        raise Exception(f"Failed to connect to {endpoint} after {max_retries} attempts")

    def create_match(self, game_type: str, num_games: int, multiplayer: bool = False) -> str:
        response = self._make_request('/new-match', {
            'game-type': game_type,
            'num-games': str(num_games),
            'multi-player': 'True' if multiplayer else 'False'
        })

        if 'match-id' not in response:
            print(f"❌ Server response missing 'match-id'. Response: {response}")
            raise KeyError(f"Server response missing 'match-id'. Got: {response}")

        return response['match-id']

    def join_match(self, match_id: str) -> dict:
        response = self._make_request('/join-match', {
            'match-id': match_id
        })
        return response

    def get_game_state(self, match_id: str, game_index: int) -> dict:
        return self._make_request('/game-state-in-match', {
            'match-id': match_id,
            'game-index': str(game_index)
        })

    def get_match_state(self, match_id: str) -> dict:
        return self._make_request('/match-state', {
            'match-id': match_id
        })

    def make_move(self, match_id: str, player: str, move: Any) -> bool:
        move_str = move if isinstance(move, str) else json.dumps(move)

        self._make_request('/make-move-in-match', {
            'match-id': match_id,
            'player': player,
            'move': move_str
        })
        return True

print("✅ GameClient loaded")


def play_game(solver, base_url: str, token: str, game_type: str, game_class,
              multiplayer: bool = False, match_id: Optional[str] = None, 
              num_games: int = 1, debug: bool = False, verbose: bool = True) -> Tuple:
    client = GameClient(base_url, token, debug=debug)

    if match_id is None:
        if verbose:
            print(f"🎮 Creating new match: {num_games} x {game_type}")
        match_id = client.create_match(game_type, num_games, multiplayer)
        if verbose:
            print(f"   Match ID: {match_id}")

    if verbose:
        print(f"🔗 Joining match {match_id}...")
    match = client.join_match(match_id)
    player = match['player']
    num_games = match.get('num-games', num_games)
    if verbose:
        print(f"   You are player: {player}")

    game_state = client.get_game_state(match_id, 0)
    if game_state['status'] == 'waiting':
        if verbose:
            print("⏳ Waiting for opponent to join...")
        while game_state['status'] == 'waiting':
            time.sleep(2)
            game_state = client.get_game_state(match_id, 0)

    all_results = []
    wins = losses = draws = 0

    while True:
        match_state = client.get_match_state(match_id)
        if match_state['status'] != 'in_progress':
            break
        game_num = match_state['current-game-index']

        if verbose:
            print(f"\n{'='*50}")
            print(f"🎮 GAME {game_num + 1}/{num_games}")
            print(f"{'='*50}\n")

        game_state = client.get_game_state(match_id, game_num)
        if 'my-player' in game_state:
            player = game_state['my-player']
        
        game = game_class(game_state['state'], game_state['status'], game_state['player'], player)

        while game_state['status'] != 'complete':
            game_state = client.get_game_state(match_id, game_num)
            if 'my-player' in game_state:
                player = game_state['my-player']
            if 'winner' in game_state:
                break

            game = game_class(game_state['state'], game_state['status'], game_state['player'], player)
            if game.is_terminal():
                break

            if verbose:
                game.print_state()

            if game.current_player == player:
                if verbose:
                    print(f"🤔 Your turn (Player {player})...")
                try:
                    move = solver(game)
                    if verbose:
                        print(f"   Move: {move}")
                    client.make_move(match_id, player, move)
                except Exception as e:
                    print(f"❌ Error in solver: {e}")
                    import traceback
                    traceback.print_exc()
                    break
            else:
                if verbose:
                    print(f"⏳ Waiting for opponent (Player {game.current_player})...")
                time.sleep(2)

        if verbose:
            game.print_state()
            print("=" * 40)

        winner = game_state.get('winner')
        if winner == '-':
            if verbose:
                print("🤝 DRAW!")
            result = 'draw'
            draws += 1
        elif winner == player:
            if verbose:
                print("🎉 You WON!")
            result = 'win'
            wins += 1
        else:
            if verbose:
                print("😞 You LOST")
            result = 'loss'
            losses += 1

        all_results.append((result, player, winner))

        if verbose and num_games > 1:
            print(f"\n📊 Record: {wins}W - {losses}L - {draws}D")

    return {
        'wins': wins, 'losses': losses, 'draws': draws,
        'total_games': num_games,
        'win_rate': wins / num_games if num_games > 0 else 0,
        'player': player, 'match_id': match_id
    }, all_results

print("✅ play_game loaded")

✅ GameClient loaded
✅ play_game loaded


---
## Section 3: Game State Class

**Run this cell (no changes needed)**

In [3]:
class UnoGame:
    """Represents UNO game state."""

    def __init__(self, state: str, status: str, current_player: str, my_player: str):
        self.state_str = state
        self.status = status
        self.current_player = current_player
        self.my_player = my_player
        self._state = None

    @property
    def state(self) -> Dict:
        if self._state is None:
            self._state = json.loads(self.state_str)
        return self._state

    def get_my_hand(self) -> List[Dict]:
        return self.state['hands'].get(self.my_player, [])

    def get_hand_sizes(self) -> Dict[str, int]:
        hands = self.state.get('hands', {})
        return {player: len(hand) for player, hand in hands.items()}

    def get_current_color(self) -> str:
        return self.state.get('current_color', '')

    def get_top_card(self) -> Dict:
        discard_pile = self.state.get('discard_pile', [])
        return discard_pile[-1] if discard_pile else {}

    def get_discard_pile(self) -> List[Dict]:
        """Get the full discard pile (all cards that have been played)."""
        return self.state.get('discard_pile', [])

    def get_discard_pile_size(self) -> int:
        """Get the number of cards in the discard pile."""
        return len(self.state.get('discard_pile', []))

    def is_terminal(self) -> bool:
        return self.status == 'complete'

    def _can_play_card(self, card: Dict, top_card: Dict, current_color: str) -> bool:
        if not top_card:
            return True
        if card.get('type') == 'wild':
            return True
        if card.get('color') == current_color:
            return True
        if card.get('value') == top_card.get('value'):
            return True
        return False

    def get_valid_moves(self) -> List[Dict]:
        if self.current_player != self.my_player:
            return []
        
        hand = self.get_my_hand()
        top_card = self.get_top_card()
        current_color = self.get_current_color()
        valid_moves = []
        
        for i, card in enumerate(hand):
            if self._can_play_card(card, top_card, current_color):
                move = {'type': 'play', 'card_index': i, 'card': card, 'call_uno': len(hand) == 2}
                if card.get('type') == 'wild':
                    for color in ['red', 'blue', 'green', 'yellow']:
                        color_move = move.copy()
                        color_move['color_choice'] = color
                        valid_moves.append(color_move)
                else:
                    valid_moves.append(move)
        
        valid_moves.append({'type': 'draw', 'count': 1})
        return valid_moves

    def print_state(self):
        print(f"\n{'='*50}")
        print(f"Current Turn: Player {self.current_player}")
        print(f"Current Color: {self.get_current_color().upper()}")
        top_card = self.get_top_card()
        if top_card:
            print(f"Top Card: {top_card.get('color')} {top_card.get('value')}")
        hand_sizes = self.get_hand_sizes()
        print("\nHand Sizes:")
        for p in sorted(hand_sizes.keys()):
            if p != self.my_player:
                print(f"  Player {p}: {hand_sizes[p]} cards")
        my_hand = self.get_my_hand()
        print(f"\nYour Hand ({len(my_hand)} cards):")
        for i, card in enumerate(my_hand):
            print(f"  {i}: {card.get('color')} {card.get('value')}")
        print('='*50)

print("✅ UnoGame class loaded")

✅ UnoGame class loaded


---
## Section 4: Manual solver

**Run this cell (no changes needed)**

In [4]:
def manual_player_solver(game: UnoGame) -> Dict:
    """
    Interactive manual player - YOU choose your moves!
    """
    game.print_state()
    
    valid_moves = game.get_valid_moves()
    
    if not valid_moves:
        print("No valid moves!")
        return {'type': 'draw', 'count': 1}
    
    print(f"\n🎮 YOUR TURN (Player {game.my_player})!")
    print("\nValid moves:")
    
    move_list = []
    move_idx = 0
    
    # Display playable cards
    play_moves = [m for m in valid_moves if m.get('type') == 'play']
    if play_moves:
        print("\n🎴 Cards you can play:")
        for move in play_moves:
            card_idx = move.get('card_index')
            card = move.get('card', {})
            color = card.get('color', '?').upper()
            value = card.get('value', '?')
            
            if card.get('type') == 'wild':
                choice = move.get('color_choice', 'red').upper()
                print(f"  {move_idx}: Play WILD as {choice}")
            else:
                print(f"  {move_idx}: Play {color} {value} (card #{card_idx})")
            
            move_list.append(move)
            move_idx += 1
    
    # Display draw option
    print(f"\n  {move_idx}: Draw a card")
    move_list.append({'type': 'draw', 'count': 1})
    
    # Get player choice
    while True:
        try:
            choice = input(f"\nEnter move number (0-{len(move_list)-1}, or 'q' to quit): ").strip()
            
            if choice.lower() == 'q':
                raise KeyboardInterrupt()
            
            idx = int(choice)
            if 0 <= idx < len(move_list):
                selected_move = move_list[idx]
                
                # If it's a play move with a wild card, ask for color choice
                if selected_move.get('type') == 'play' and selected_move.get('card', {}).get('type') == 'wild':
                    print("\nWild card color choices:")
                    colors = ['red', 'blue', 'green', 'yellow']
                    for i, c in enumerate(colors):
                        print(f"  {i}: {c.upper()}")
                    
                    while True:
                        try:
                            color_choice = input("Choose color (0-3): ").strip()
                            color_idx = int(color_choice)
                            if 0 <= color_idx < len(colors):
                                selected_move['color_choice'] = colors[color_idx]
                                break
                            else:
                                print(f"❌ Invalid choice! Enter 0-{len(colors)-1}")
                        except ValueError:
                            print("❌ Invalid input! Enter a number.")
                
                return selected_move
            else:
                print(f"❌ Invalid index! Choose 0-{len(move_list)-1}")
        
        except ValueError:
            print("❌ Invalid input! Enter a number or 'q' to quit.")
        except KeyboardInterrupt:
            print("\n👋 Thanks for playing!")
            raise

print("✅ Manual player solver loaded")

✅ Manual player solver loaded


---
## Section 5: YOUR SOLVER IMPLEMENTATION

**⭐ THIS IS WHERE YOU WRITE YOUR CODE! ⭐**

### Available Methods

```python
game.get_my_hand()                        # List of cards in your hand
game.get_hand_sizes()                     # Dict of player hand sizes
game.get_current_color()                  # Current active color
game.get_top_card()                       # Card on top of discard pile
game.get_discard_pile()                   # Full discard pile history (all played cards)
game.get_discard_pile_size()              # Number of cards in discard pile
game.get_valid_moves()                    # All valid moves you can make
game.is_terminal()                        # Whether game is finished
game.print_state()                        # Print current game state
```

### Move Format
- Play a card: `{'type': 'play', 'card_index': i, 'card': {...}, 'color_choice': 'red'}`
  - `card_index`: Index of card in your hand (0-indexed)
  - `color_choice`: Only required for Wild cards (one of: 'red', 'blue', 'green', 'yellow')
- Draw a card: `{'type': 'draw', 'count': 1}`

### Card Information
Card dict contains: `{'color': 'red', 'value': '5', 'type': 'normal'}`
- Colors: 'red', 'blue', 'green', 'yellow'
- Values: '0'-'9', 'skip', 'reverse', 'draw2'
- Types: 'normal', 'action', 'wild'

### Strategic Tips
- **Discard Pile History**: Use `game.get_discard_pile()` to see all cards that have been played. This allows you to implement card-counting strategies!
- **Example**: Count which high cards are still in the deck vs. already played to make better decisions
- **Hand Tracking**: Use `game.get_hand_sizes()` to track how many cards opponents have

In [5]:
import math
import random

def my_agent(game: "UnoGame") -> dict:
    """
    Super UNO AI: combines
    - Agent 1: probability + card counting + aggression factor
    - Agent 2: next-player awareness, game states, color selection, hand shaping
    """
    # 1. Extract Game States (Common)
    my_hand = game.get_my_hand()
    hand_sizes = game.get_hand_sizes()
    discard_pile = game.get_discard_pile()
    valid_moves = game.get_valid_moves()
    current_color = game.get_current_color()
    top_card = game.get_top_card()
    my_player = game.my_player

    colors = ['red', 'blue', 'green', 'yellow']
    
    play_moves = [m for m in valid_moves if m.get('type') == 'play']
    draw_moves = [m for m in valid_moves if m.get('type') == 'draw']
    
    if not play_moves:
        if draw_moves:
            return draw_moves[0]
        return {'type': 'draw', 'count': 1}

    # 2. Hand and discard statistics (Common)
    current_size = len(my_hand)

    ## Count colors in my hand (Common)
    hand_color_count = {c: 0 for c in colors}
    for card in my_hand:
        col = card.get('color')
        if col in hand_color_count:
            hand_color_count[col] += 1

    ## Count colors in discard pile (Common)
    discard_color_count = {c: 0 for c in colors}
    for card in discard_pile:
        col = card.get('color')
        if col in discard_color_count:
            discard_color_count[col] += 1

    # Statistics on opponent hand sizes (Oliver)
    opp_sizes = [size for p, size in hand_sizes.items() if p != my_player]
    opp_min = min(opp_sizes) if opp_sizes else 10
    opp_avg = sum(opp_sizes) / len(opp_sizes) if opp_sizes else current_size

    # Aggression factor (Oliver)
    size_delta = current_size - opp_avg
    if size_delta >= 3:
        aggression_factor = 1.25   # behind: push harder
    elif size_delta <= -3:
        aggression_factor = 0.8    # ahead: safer
    else:
        aggression_factor = 1.0

    # 3. Next-player awareness (Keagan)
    def get_next_players():
        players = sorted(hand_sizes.keys())
        if my_player not in players or len(players) == 0:
            return None, None

        idx = players.index(my_player)
        next_idx = (idx + 1) % len(players)
        if len(players) > 2:
            next_next_idx = (idx + 2) % len(players)
        else:
            next_next_idx = idx  # in 2-player game, skipping returns to us

        return players[next_idx], players[next_next_idx]

    next_player, next_next_player = get_next_players()
    my_size = current_size
    next_size = hand_sizes.get(next_player, 0) if next_player is not None else 0

    # 4. Game state classification (Keagan)
    def assess_game_state(my_size, next_size, low_threshold):
        my_low = my_size <= low_threshold
        next_low = next_size <= low_threshold

        if not my_low and not next_low:
            return "default"   # both high
        elif my_low and not next_low:
            return "winning"   # we are low, they are high
        elif not my_low and next_low:
            return "threat"    # they are low, we are high
        else:
            return "luck"      # both low

    game_state = assess_game_state(my_size, next_size, low_threshold=2)

    # 5. Approximate card-counting / probability (Oliver)
    BASE_COLOR_COUNT = 25  # rough per-color count; relative ratios matter

    seen_color_count = {
        c: hand_color_count[c] + discard_color_count[c] for c in colors
    }
    unseen_color_count = {
        c: max(BASE_COLOR_COUNT - seen_color_count[c], 0) for c in colors
    }
    total_unseen = sum(unseen_color_count.values())
    if total_unseen <= 0:
        total_unseen = len(colors)
        unseen_color_count = {c: 1 for c in colors}

    def prob_short_opponent_has_color(color: str) -> float:
        """Approximate P(shortest opponent can play color)."""
        if not opp_sizes or color not in colors:
            return 0.5
        k = opp_min
        if k <= 0:
            return 0.0
        unseen_c = unseen_color_count.get(color, 0)
        if unseen_c <= 0:
            return 0.0
        p_c = unseen_c / total_unseen
        return 1.0 - math.pow(1.0 - p_c, k)

    # 6. Wild color choice helpers (Keagan, adapted for Oliver probability)
    def choose_color_for_wild():
        best_color = colors[0]
        best_score = -1e9

        # Any opponent with very low cards
        low_opponent_exists = any(
            (pid != my_player) and (size <= 2)
            for pid, size in hand_sizes.items()
        )

        for col in colors:
            my_count = hand_color_count.get(col, 0)
            disc_count = discard_color_count.get(col, 0)
            score = 0.8 * my_count - 0.2 * disc_count

            # Avoid keeping current color when someone is low (Keagan)
            if low_opponent_exists and col == current_color:
                score -= 0.5

            # Avoid colors that opponents are likely to have (Oliver)
            if low_opponent_exists:
                color_hazard = prob_short_opponent_has_color(col)
                score -= 0.8 * color_hazard  # penalty if opponents likely hold it

            if score > best_score:
                best_score = score
                best_color = col

        return best_color

    # Wild card choice helper for edge case (Keagan)
    def choose_color_for_wild_edge(card_index: int):
        """
        If playing wild and we will have 1 card left,
        choose that card's color if possible.
        """
        hand_after = my_hand[:card_index] + my_hand[card_index + 1:]
        if len(hand_after) == 1:
            last_card = hand_after[0]
            last_color = last_card.get('color')
            if last_color in colors:
                return last_color
        return choose_color_for_wild()

    # Next player has 1 card (Keagan)
    # ----------------------------------------------------------
    if next_player is not None and next_size == 1:
        # Try everything to mess them up:
        # priority: draw4/draw2 > skip/reverse > wild color change
        draw_power_moves = []
        deny_turn_moves = []
        wild_moves = []

        for move in play_moves:
            card = move.get('card', {})
            ctype = card.get('type')
            value = card.get('value')

            if ctype == 'wild' and str(value).lower() in ['wild_draw4', 'draw4', 'draw4_wild']:
                draw_power_moves.append(move)
            elif ctype == 'action' and value == 'draw2':
                draw_power_moves.append(move)
            elif ctype == 'action' and value in ('skip', 'reverse'):
                deny_turn_moves.append(move)
            elif ctype == 'wild':
                wild_moves.append(move)

        chosen = None
        if draw_power_moves:
            chosen = draw_power_moves[0]
        elif deny_turn_moves:
            chosen = deny_turn_moves[0]
        elif wild_moves:
            chosen = wild_moves[0]

        if chosen is not None:
            card = chosen.get('card', {})
            if card.get('type') == 'wild' and 'color_choice' not in chosen:
                idx = chosen.get('card_index', 0)
                chosen = dict(chosen)
                if my_size == 2:
                    chosen['color_choice'] = choose_color_for_wild_edge(idx)
                else:
                    chosen['color_choice'] = choose_color_for_wild()
            return chosen

    # 8. Move evaluation (Hybrid)
    def score_move(move: dict) -> float:
        card = move.get('card', {})
        ctype = card.get('type')   
        value = card.get('value')
        color = card.get('color')
        idx = move.get('card_index', None)

        new_size = current_size - 1
        score = 0.0

        # 8.2 Base card-type preference (Keagan, adapted)
        if ctype in ('number'):
            base = 1.0
        elif ctype == 'action':
            if value == 'draw2':
                base = 2.0
            elif value in ('skip', 'reverse'):
                base = 1.8
            else:
                base = 1.5
        elif ctype == 'wild':
            if value == 'wild_draw4':
                base = 2.3
            else:
                base = 2.0
        else:
            base = 1.0
        score += base

        # 8.3 Determine the post-move active color
        if ctype == 'wild':
            chosen_color = move.get('color_choice')
            # If color_choice missing (just in case), infer a smart one
            if chosen_color not in colors:
                if my_size == 2 and idx is not None:
                    chosen_color = choose_color_for_wild_edge(idx)
                else:
                    chosen_color = choose_color_for_wild()
        else:
            chosen_color = color

        if chosen_color not in colors:
            chosen_color = current_color if current_color in colors else None

        my_color_before = hand_color_count.get(chosen_color, 0) if chosen_color in colors else 0
        my_color_after = my_color_before
        if ctype != 'wild' and chosen_color in colors and color == chosen_color:
            my_color_after = max(my_color_before - 1, 0)

        played_for_color = discard_color_count.get(chosen_color, 0) if chosen_color in colors else 0

        # 8.4 Opponent danger / color probability (Oliver)
        if chosen_color in colors:
            color_hazard = prob_short_opponent_has_color(chosen_color)
        else:
            color_hazard = 0.5

        # Prefer colors opponents are less likely to hold when someone is low (Oliver)
        if opp_min <= 2:
            score += aggression_factor * 25.0 * (1.0 - color_hazard)
        elif opp_min <= 3:
            score += aggression_factor * 12.0 * (1.0 - color_hazard)
        elif opp_min <= 4:
            score += aggression_factor * 5.0 * (1.0 - color_hazard)

        # 8.5 Punish short opponents with specials (Oliver)
        if opp_min <= 2:
            if ctype == 'action':
                if value in ['skip', 'reverse']:
                    score += aggression_factor * 20.0
                elif value == 'draw2':
                    score += aggression_factor * 28.0
            if ctype == 'wild' and str(value).lower() in ['draw4', 'wild_draw4', 'draw4_wild']:
                score += aggression_factor * 35.0
        elif opp_min <= 3:
            if ctype in ['action', 'wild']:
                score += aggression_factor * 10.0

        # 8.6 Hand-shaping / color distribution (Keagan)
        hand_after = my_hand
        if idx is not None and 0 <= idx < len(my_hand):
            hand_after = my_hand[:idx] + my_hand[idx + 1:]

        size_after = len(hand_after)
        colors_after = [
            c.get('color')
            for c in hand_after
            if c.get('color') in colors and c.get('type') != 'wild'
        ]
        distinct_colors_after = len(set(colors_after))

        # Prefer fewer distinct colors in hand (easier to play)
        if size_after >= 7:
            score += (3 - distinct_colors_after) * 0.2
        elif 4 <= size_after <= 6:
            target = 2
            score += (target - abs(distinct_colors_after - target)) * 0.1
        elif size_after == 2:
            if distinct_colors_after == 1:
                score += 2.0
            else:
                score -= 1.0
        else:
            score += (3 - distinct_colors_after) * 0.3

        # Slight additional tweak for size_after == 2
        if size_after == 2:
            if distinct_colors_after == 1:
                score += 2.0
            else:
                score -= 1.0

        # 8.7 Color choice preferences (Hybrid)
        if ctype != 'wild':
            if color in colors:
                my_col = hand_color_count.get(color, 0)
                disc_col = discard_color_count.get(color, 0)
                score += 0.3 * my_col - 0.05 * disc_col
        else:
            # wild: prefer best future color
            pref_col = choose_color_for_wild()
            my_col = hand_color_count.get(pref_col, 0)
            disc_col = discard_color_count.get(pref_col, 0)
            score += 0.25 * my_col - 0.05 * disc_col

        # 8.8 Card-counting nudge: colors heavily played are safer to push (Oliver)
        if chosen_color in colors:
            score += 0.3 * played_for_color

        # 8.9 Late-game preferences (Hybrid)
        if current_size <= 3:
            # Avoid being stuck with specials at the very end
            if ctype in ['action', 'wild']:
                score += 8.0

            # Dump high number cards late
            try:
                num_val = int(value)
                score += 0.5 * num_val
            except (TypeError, ValueError):
                pass

            # Try to keep remaining hand mostly in one strong color
            if chosen_color in colors:
                score += 2.0 * my_color_after

        # 8.10 Game-state specific tweaks (Keagan)
        if game_state == "default":
            if size_after > 3:
                # Early/mid game: save powerful actions
                if ctype in ("wild", "action"):
                    score -= 10.0
                    if value in ("draw2", "wild_draw4", "draw4"):
                        score -= 5.0
                if ctype in ("normal", "number"):
                    score += 0.8
            else:
                if ctype in ("normal", "number"):
                    score += 0.5
                if ctype == 'action':
                    score += 0.5

        elif game_state == "winning":
            # Beeline to win
            if size_after == 1:
                score += 6.0
            elif size_after == 2:
                score += 3.0
            if ctype in ("normal", "number"):
                score += 1.0
            if ctype == "wild" and size_after <= 1:
                score += 3.0

        elif game_state == "threat":
            # Opponent close to winning -> value disruption
            if ctype == "action":
                if value == "draw2":
                    score += 8.0
                elif value in ("skip", "reverse"):
                    score += 6.0
                else:
                    score += 4.0
            elif ctype == "wild":
                if str(value).lower() in ['wild_draw4', 'draw4', 'draw4_wild']:
                    score += 10.0
                else:
                    score += 6.0
            elif ctype in ("normal", "number"):
                score -= 5.0

        elif game_state == "luck":
            # Both low: want to win but avoid setting them up
            if ctype == 'wild':
                if str(value).lower() in ['wild_draw4', 'draw4', 'draw4_wild']:
                    score += 6.0
            if ctype == "action":
                if value == "draw2":
                    score += 6.0
                elif value in ("skip", "reverse"):
                    score += 4.0
            if ctype in ("normal", "number"):
                score += 0.5

        # 8.11 Keep current color slightly if it's already on table
        if chosen_color == current_color and chosen_color in colors:
            score += 2.0

        # 8.12 Number card dump preference (Oliver)
        if ctype in ("normal", "number"):
            try:
                num_val = int(value)
                score += 0.4 * num_val
            except (TypeError, ValueError):
                pass

        # 8.13 Extra wild logic (Oliver)
        if ctype == 'wild':
            score += 8.0  # generic power
            score += 4.0 * my_color_before
            score += 0.4 * played_for_color
            if current_size >= 6 and opp_min > 3:
                score -= 15.0  # early-game wild penalty
            if current_size <= 3:
                score += 15.0  # late-game finisher
            if str(value).lower() in ['draw4', 'wild_draw4', 'draw4_wild']:
                score += aggression_factor * 10.0

        # Tiny random noise to break ties
        score += random.random() * 0.01

        return score

    # ----------------------------------------------------------
    # 9. Choose best move
    # ----------------------------------------------------------
    best_score = None
    best_moves = []

    for move in play_moves:
        s = score_move(move)
        if best_score is None or s > best_score:
            best_score = s
            best_moves = [move]
        elif s == best_score:
            best_moves.append(move)

    chosen_move = random.choice(best_moves)

    # ----------------------------------------------------------
    # 10. Ensure wild color_choice if needed
    # ----------------------------------------------------------
    if chosen_move.get('type') == 'play':
        card = chosen_move.get('card', {})
        if card.get('type') == 'wild' and 'color_choice' not in chosen_move:
            idx = chosen_move.get('card_index', 0)
            chosen_move = dict(chosen_move)
            if current_size == 2:
                chosen_move['color_choice'] = choose_color_for_wild_edge(idx)
            else:
                chosen_move['color_choice'] = choose_color_for_wild()

    # UNO call logic
    if chosen_move.get('type') == 'play' and current_size - 1 == 1:
        chosen_move['call_uno'] = True

    return chosen_move

print("✅ Super solver defined - my_agent() now uses hybrid strategy")

✅ Super solver defined - my_agent() now uses hybrid strategy


---
## Section 6: Play the Game!

**Update STUDENT_TOKEN below and run to play**

You can choose which solver to use:
- `my_agent` - Your AI implementation (default)
- `manual_player_solver` - Interactive manual play

In [6]:
STUDENT_TOKEN = 'K2'
SOLVER = my_agent  # Change to manual_player_solver to play manually!
MULTIPLAYER = False
MATCH_ID = None
NUM_GAMES = 10

try:
    stats, results = play_game(
        solver=SOLVER,
        base_url=BASE_URL,
        token=STUDENT_TOKEN,
        game_type='uno4',
        game_class=UnoGame,
        multiplayer=MULTIPLAYER,
        num_games=NUM_GAMES,
        match_id=MATCH_ID,
        verbose=True
    )

    print("\n📊 Summary:")
    print(f"   Record: {stats['wins']}W - {stats['losses']}L - {stats['draws']}D")
    print(f"   Win Rate: {stats['win_rate']*100:.1f}%")

except Exception as e:
    print(f"❌ Game error: {e}")
    import traceback
    traceback.print_exc()

🎮 Creating new match: 10 x uno4
   Match ID: 2759
🔗 Joining match 2759...
   You are player: 1

🎮 GAME 1/10


Current Turn: Player 1
Current Color: GREEN
Top Card: green 9

Hand Sizes:
  Player 2: 7 cards
  Player 3: 7 cards
  Player 4: 7 cards

Your Hand (7 cards):
  0: blue draw2
  1: wild wild_draw4
  2: wild wild
  3: red 8
  4: yellow skip
  5: blue 5
  6: blue 3
🤔 Your turn (Player 1)...
   Move: {'type': 'play', 'card_index': 1, 'card': {'color': 'wild', 'value': 'wild_draw4', 'type': 'wild'}, 'call_uno': False, 'color_choice': 'blue'}

Current Turn: Player 1
Current Color: YELLOW
Top Card: wild wild

Hand Sizes:
  Player 2: 11 cards
  Player 3: 6 cards
  Player 4: 6 cards

Your Hand (6 cards):
  0: blue draw2
  1: wild wild
  2: red 8
  3: yellow skip
  4: blue 5
  5: blue 3
🤔 Your turn (Player 1)...
   Move: {'type': 'play', 'card_index': 1, 'card': {'color': 'wild', 'value': 'wild', 'type': 'wild'}, 'call_uno': False, 'color_choice': 'blue'}

Current Turn: Player 2
Current Co


Current Turn: Player 1
Current Color: YELLOW
Top Card: yellow draw2

Hand Sizes:
  Player 2: 10 cards
  Player 3: 5 cards
  Player 4: 5 cards

Your Hand (1 cards):
  0: blue draw2
🤔 Your turn (Player 1)...
   Move: {'type': 'play', 'card_index': 0, 'card': {'color': 'blue', 'value': 'draw2', 'type': 'action'}, 'call_uno': False}

Current Turn: Player 1
Current Color: YELLOW
Top Card: yellow draw2

Hand Sizes:
  Player 2: 10 cards
  Player 3: 5 cards
  Player 4: 5 cards

Your Hand (1 cards):
  0: blue draw2
🎉 You WON!

📊 Record: 1W - 0L - 0D

🎮 GAME 2/10


Current Turn: Player 1
Current Color: YELLOW
Top Card: yellow 1

Hand Sizes:
  Player 2: 7 cards
  Player 3: 7 cards
  Player 4: 7 cards

Your Hand (7 cards):
  0: yellow reverse
  1: green 4
  2: green skip
  3: blue 0
  4: blue reverse
  5: green 3
  6: blue 4
🤔 Your turn (Player 1)...
   Move: {'type': 'play', 'card_index': 0, 'card': {'color': 'yellow', 'value': 'reverse', 'type': 'action'}, 'call_uno': False}

Current Turn: Play


Current Turn: Player 1
Current Color: GREEN
Top Card: green 8

Hand Sizes:
  Player 2: 4 cards
  Player 3: 3 cards
  Player 4: 4 cards

Your Hand (7 cards):
  0: green 4
  1: blue 0
  2: green 3
  3: blue 4
  4: yellow 4
  5: green draw2
  6: yellow 5
🤔 Your turn (Player 1)...
   Move: {'type': 'play', 'card_index': 0, 'card': {'color': 'green', 'value': 4, 'type': 'number'}, 'call_uno': False}

Current Turn: Player 3
Current Color: GREEN
Top Card: green 4

Hand Sizes:
  Player 2: 4 cards
  Player 3: 3 cards
  Player 4: 5 cards

Your Hand (6 cards):
  0: blue 0
  1: green 3
  2: blue 4
  3: yellow 4
  4: green draw2
  5: yellow 5
⏳ Waiting for opponent (Player 3)...

Current Turn: Player 2
Current Color: YELLOW
Top Card: yellow 4

Hand Sizes:
  Player 2: 4 cards
  Player 3: 2 cards
  Player 4: 5 cards

Your Hand (6 cards):
  0: blue 0
  1: green 3
  2: blue 4
  3: yellow 4
  4: green draw2
  5: yellow 5
⏳ Waiting for opponent (Player 2)...

Current Turn: Player 1
Current Color: YELLOW


Current Turn: Player 1
Current Color: RED
Top Card: red 6

Hand Sizes:
  Player 1: 3 cards
  Player 2: 5 cards
  Player 4: 6 cards

Your Hand (8 cards):
  0: green 8
  1: yellow 1
  2: green 3
  3: green reverse
  4: blue skip
  5: yellow 3
  6: blue 1
  7: wild wild_draw4
⏳ Waiting for opponent (Player 1)...

Current Turn: Player 1
Current Color: RED
Top Card: red reverse

Hand Sizes:
  Player 1: 4 cards
  Player 2: 4 cards
  Player 4: 6 cards

Your Hand (8 cards):
  0: green 8
  1: yellow 1
  2: green 3
  3: green reverse
  4: blue skip
  5: yellow 3
  6: blue 1
  7: wild wild_draw4
⏳ Waiting for opponent (Player 1)...

Current Turn: Player 1
Current Color: YELLOW
Top Card: yellow reverse

Hand Sizes:
  Player 1: 5 cards
  Player 2: 4 cards
  Player 4: 5 cards

Your Hand (8 cards):
  0: green 8
  1: yellow 1
  2: green 3
  3: green reverse
  4: blue skip
  5: yellow 3
  6: blue 1
  7: wild wild_draw4
⏳ Waiting for opponent (Player 1)...

Current Turn: Player 3
Current Color: YELLOW



Current Turn: Player 1
Current Color: GREEN
Top Card: green 7

Hand Sizes:
  Player 2: 4 cards
  Player 3: 6 cards
  Player 4: 3 cards

Your Hand (5 cards):
  0: yellow 3
  1: yellow 9
  2: yellow 6
  3: blue 3
  4: green 3
🤔 Your turn (Player 1)...
   Move: {'type': 'play', 'card_index': 4, 'card': {'color': 'green', 'value': 3, 'type': 'number'}, 'call_uno': False}

Current Turn: Player 3
Current Color: GREEN
Top Card: green 3

Hand Sizes:
  Player 2: 4 cards
  Player 3: 6 cards
  Player 4: 4 cards

Your Hand (4 cards):
  0: yellow 3
  1: yellow 9
  2: yellow 6
  3: blue 3
⏳ Waiting for opponent (Player 3)...

Current Turn: Player 3
Current Color: RED
Top Card: red reverse

Hand Sizes:
  Player 2: 4 cards
  Player 3: 5 cards
  Player 4: 3 cards

Your Hand (4 cards):
  0: yellow 3
  1: yellow 9
  2: yellow 6
  3: blue 3
⏳ Waiting for opponent (Player 3)...

Current Turn: Player 2
Current Color: RED
Top Card: red reverse

Hand Sizes:
  Player 2: 4 cards
  Player 3: 6 cards
  Player 4:

...


Current Turn: Player 1
Current Color: YELLOW
Top Card: yellow 1

Hand Sizes:
  Player 1: 8 cards
  Player 2: 4 cards
  Player 4: 7 cards

Your Hand (6 cards):
  0: yellow 0
  1: red draw2
  2: blue 9
  3: green 2
  4: blue 4
  5: red reverse
⏳ Waiting for opponent (Player 1)...

Current Turn: Player 3
Current Color: BLUE
Top Card: blue 7

Hand Sizes:
  Player 1: 7 cards
  Player 2: 3 cards
  Player 4: 7 cards

Your Hand (6 cards):
  0: yellow 0
  1: red draw2
  2: blue 9
  3: green 2
  4: blue 4
  5: red reverse
🤔 Your turn (Player 3)...
   Move: {'type': 'play', 'card_index': 2, 'card': {'color': 'blue', 'value': 9, 'type': 'number'}, 'call_uno': False}

Current Turn: Player 2
Current Color: BLUE
Top Card: blue skip

Hand Sizes:
  Player 1: 7 cards
  Player 2: 3 cards
  Player 4: 6 cards

Your Hand (5 cards):
  0: yellow 0
  1: red draw2
  2: green 2
  3: blue 4
  4: red reverse
⏳ Waiting for opponent (Player 2)...

Current Turn: Player 3
Current Color: BLUE
Top Card: blue skip

Hand


Current Turn: Player 4
Current Color: BLUE
Top Card: blue 5

Hand Sizes:
  Player 1: 6 cards
  Player 2: 11 cards
  Player 3: 6 cards

Your Hand (6 cards):
  0: blue 3
  1: wild wild
  2: wild wild
  3: yellow 8
  4: yellow 3
  5: blue reverse
🤔 Your turn (Player 4)...
   Move: {'type': 'play', 'card_index': 0, 'card': {'color': 'blue', 'value': 3, 'type': 'number'}, 'call_uno': False}

Current Turn: Player 4
Current Color: BLUE
Top Card: wild wild_draw4

Hand Sizes:
  Player 1: 5 cards
  Player 2: 10 cards
  Player 3: 10 cards

Your Hand (5 cards):
  0: wild wild
  1: wild wild
  2: yellow 8
  3: yellow 3
  4: blue reverse
🤔 Your turn (Player 4)...
   Move: {'type': 'play', 'card_index': 0, 'card': {'color': 'wild', 'value': 'wild', 'type': 'wild'}, 'call_uno': False, 'color_choice': 'blue'}

Current Turn: Player 4
Current Color: BLUE
Top Card: blue 7

Hand Sizes:
  Player 1: 4 cards
  Player 2: 12 cards
  Player 3: 9 cards

Your Hand (4 cards):
  0: wild wild
  1: yellow 8
  2: yell


Current Turn: Player 4
Current Color: RED
Top Card: red 7

Hand Sizes:
  Player 1: 2 cards
  Player 2: 4 cards
  Player 3: 5 cards

Your Hand (3 cards):
  0: yellow 3
  1: blue 1
  2: green draw2
🤔 Your turn (Player 4)...
   Move: {'type': 'draw', 'count': 1}

Current Turn: Player 1
Current Color: RED
Top Card: red skip

Hand Sizes:
  Player 1: 2 cards
  Player 2: 4 cards
  Player 3: 4 cards

Your Hand (4 cards):
  0: yellow 3
  1: blue 1
  2: green draw2
  3: red draw2
⏳ Waiting for opponent (Player 1)...

Current Turn: Player 4
Current Color: RED
Top Card: red skip

Hand Sizes:
  Player 1: 3 cards
  Player 2: 4 cards
  Player 3: 4 cards

Your Hand (4 cards):
  0: yellow 3
  1: blue 1
  2: green draw2
  3: red draw2
🤔 Your turn (Player 4)...
   Move: {'type': 'play', 'card_index': 3, 'card': {'color': 'red', 'value': 'draw2', 'type': 'action'}, 'call_uno': False}

Current Turn: Player 1
Current Color: RED
Top Card: red draw2

Hand Sizes:
  Player 1: 5 cards
  Player 2: 5 cards
  Play


Current Turn: Player 1
Current Color: BLUE
Top Card: blue 7

Hand Sizes:
  Player 1: 4 cards
  Player 2: 6 cards
  Player 4: 5 cards

Your Hand (7 cards):
  0: green 8
  1: blue 5
  2: red 3
  3: blue 3
  4: green draw2
  5: green 3
  6: blue 1
⏳ Waiting for opponent (Player 1)...

Current Turn: Player 1
Current Color: BLUE
Top Card: blue 8

Hand Sizes:
  Player 1: 3 cards
  Player 2: 5 cards
  Player 4: 4 cards

Your Hand (9 cards):
  0: green 8
  1: blue 5
  2: red 3
  3: blue 3
  4: green draw2
  5: green 3
  6: blue 1
  7: wild wild_draw4
  8: red 5
⏳ Waiting for opponent (Player 1)...

Current Turn: Player 3
Current Color: GREEN
Top Card: green 8

Hand Sizes:
  Player 1: 2 cards
  Player 2: 4 cards
  Player 4: 4 cards

Your Hand (9 cards):
  0: green 8
  1: blue 5
  2: red 3
  3: blue 3
  4: green draw2
  5: green 3
  6: blue 1
  7: wild wild_draw4
  8: red 5
🤔 Your turn (Player 3)...
   Move: {'type': 'play', 'card_index': 7, 'card': {'color': 'wild', 'value': 'wild_draw4', 'typ


Current Turn: Player 3
Current Color: RED
Top Card: red 1

Hand Sizes:
  Player 1: 4 cards
  Player 2: 2 cards
  Player 4: 4 cards

Your Hand (4 cards):
  0: red 3
  1: blue 1
  2: blue 2
  3: green 1
🤔 Your turn (Player 3)...
   Move: {'type': 'play', 'card_index': 0, 'card': {'color': 'red', 'value': 3, 'type': 'number'}, 'call_uno': False}

Current Turn: Player 1
Current Color: BLUE
Top Card: blue 3

Hand Sizes:
  Player 1: 4 cards
  Player 2: 1 cards
  Player 4: 4 cards

Your Hand (3 cards):
  0: blue 1
  1: blue 2
  2: green 1
⏳ Waiting for opponent (Player 1)...

Current Turn: Player 3
Current Color: YELLOW
Top Card: wild wild_draw4

Hand Sizes:
  Player 1: 3 cards
  Player 2: 5 cards
  Player 4: 4 cards

Your Hand (3 cards):
  0: blue 1
  1: blue 2
  2: green 1
🤔 Your turn (Player 3)...
   Move: {'type': 'draw', 'count': 1}

Current Turn: Player 1
Current Color: YELLOW
Top Card: yellow 9

Hand Sizes:
  Player 1: 3 cards
  Player 2: 4 cards
  Player 4: 4 cards

Your Hand (4 card